https://towardsdatascience.com/hog-histogram-of-oriented-gradients-67ecd887675f

https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html

https://www.thepythoncode.com/article/hog-feature-extraction-in-python

In [ ]:
pip install scikit-image

In [ ]:
import matplotlib.pyplot as plt
from skimage import io
from skimage import color
from skimage.transform import resize
import math
from skimage.feature import hog
import numpy as np
import pandas as pd
from PIL import Image
from skimage import data, exposure

In [ ]:
df = pd.read_csv('data/train.csv')

1600; 256
1600/256 = 6,25

In [ ]:
image = io.imread("data/train_images/000418bfc.jpg")

In [ ]:
img = resize(color.rgb2gray(io.imread("data/train_images/000418bfc.jpg")), (64, 128))

In [ ]:
plt.imshow(image)
plt.xlim(0,1600)
plt.ylim(0,256)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

In [ ]:
plt.imshow(img)
plt.xlim(0,128)
plt.ylim(0,64)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

The hog() function takes 6 parameters as input:

image: The target image you want to apply HOG feature extraction.

orientations: Number of bins in the histogram we want to create, the original research paper used 9 bins so we will pass 9 as orientations.

pixels_per_cell: Determines the size of the cell, as we mentioned earlier, it is 8x8.

cells_per_block: Number of cells per block, will be 2x2 as mentioned previously.

visualize: A boolean whether to return the image of the HOG, we set it to True so we can show the image.

multichannel: We set it to True to tell the function that the last dimension is considered as a color channel, instead of spatial.

In [ ]:
#without resized picture

fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, channel_axis=-1)

fig, (ax1, ax2) = plt.subplots(2,1, figsize=(32, 16), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()